In [ ]:
import itertools

def generate_grids(grid, max_replacements):
    # Find indices of 0s in the grid
    zero_indices = [(i, j) for i, row in enumerate(grid) for j, val in enumerate(row) if val == 0]
    
    # Generate all possible combinations of replacements
    for r in range(min(max_replacements, len(zero_indices)) + 1):
        for indices in itertools.combinations(zero_indices, r):
            for replacements in itertools.product(range(1, 15), repeat=r):
                new_grid = [row[:] for row in grid]  # Create a deep copy of the grid
                for (i, j), value in zip(indices, replacements):
                    new_grid[i][j] = value
                yield new_grid

In [ ]:
initial_grid = [[5, 15, 15], [0, 0, 0], [15, 15, 0], [0, 0, 0], [0, 15, 15], [0, 0, 6]]
max_replacements = 2

results = list(generate_grids(initial_grid, max_replacements))
print(len(results))  #

In [ ]:
# redner video
from grid import Grid
from cart import Cart
import os
from ffmpeg import FFmpeg
import json
from utils import DIRECTION
import copy
import shutil
from tqdm import tqdm
import numpy as np
import time

root_folder = "blog_output"
os.makedirs(root_folder, exist_ok=True)


grid_to_visual = [
    "levels/1-1.json",
    "levels/1-3.json",
    "levels/1-6.json",
    "levels/1-7.json",
    "levels/1-8.json",
    "levels/1-9.json",
]

for grid_file in grid_to_visual:
    name = os.path.basename(grid_file).split(".")[0]
    output_folder = os.path.join(root_folder, name)
    # os.makedirs(output_folder, exist_ok=True)

    with open(grid_file, "r") as f:
        data = json.load(f)
    grid = data["grid"]
    destination = data["destination"]
    max_tracks = data["max_tracks"]
    carts = [
        Cart((cart["x"], cart["y"]), DIRECTION(cart["direction"]), cart["order"])
        for cart in data["carts"]
    ]
    frames = []
    start_time = time.process_time()
    for i, _g in enumerate(tqdm(generate_grids(grid, max_tracks), desc=name + " ")):
        g = Grid(_g, destination)
        frames.append(g)
        if not g.is_grid_valid():
            continue
        carts_copy = [copy.deepcopy(cart) for cart in carts]
        simulate_result = g.simulate(carts_copy)
        if simulate_result[0] == "success":
            break
    print(f"Time taken for {name}: {time.process_time() - start_time}")
    print(f"Total frames: {i}")
    os.makedirs(output_folder, exist_ok=True)
    total_frames = 200
    indices = np.round(np.linspace(0, i-1, total_frames)).astype(int)
    for i, j in enumerate(indices):
        img = frames[j].get_image()
        img.save(os.path.join(output_folder, f"{name}_{i}.png"))
    # add 10 more frames to make the video more stable
    for k in range(i, i+10):
        img = frames[-1].get_image()
        img.save(os.path.join(output_folder, f"{name}_{k}.png"))
    full_path = os.getcwd()
    ffmpeg = (
        FFmpeg()
        .option("y")
        .input(os.path.join(output_folder, f"{name}_%d.png"), framerate=20)
        .output(
            os.path.join(full_path, root_folder, f"{name}.mp4"),
            {
                "c:v": "libx265",
            },
        )
    )
    out=ffmpeg.execute()
    shutil.rmtree(output_folder)

1-1 : 1475it [00:00, 19332.09it/s]
Time taken for 1-1: 0.07502099999999956
Total frames: 1475

1-3 : 37it [00:00, 40742.78it/s]
Time taken for 1-3: 0.0036529999999999063
Total frames: 37
1-6 : 518467it [00:09, 57346.18it/s]
Time taken for 1-6: 8.802503
Total frames: 518467
1-7 : 56017it [00:00, 78190.38it/s] 
Time taken for 1-7: 0.6180190000000003
Total frames: 56017
1-8 : 76557it [00:01, 55072.22it/s]
Time taken for 1-8: 1.365589
Total frames: 76557
1-9 : 5078095it [01:51, 45740.02it/s]
Time taken for 1-9: 105.97420199999999
Total frames: 5078094